<a href="https://colab.research.google.com/github/Bast1-py/Experience-Developing-Projects/blob/main/Evaluation_and_Analysis_of_Student_Performance_with_Explainable_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://skills.network/logos/SN_web_lightmode.png"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 520px; height: auto;"></div>')

---
# 1. Introduction:
---

diera solusi yang didukung AI, memahami mengapa dan bagaimana sebuah model membuat keputusan sangatlah penting, terutama dibidang seperti education, healthcare, dan finance. Explainable AI (XAI) mempu menjebatani kesenjangan  antara model machine learning yang kompleks dan pemahaman manusia dengan menyediakan alat dan metode yang membuat proses pengambilan keputusan menjadi lebih transparan dan dapat ditafsirkan oleh semua stakeholders, termasuk developers, educators, dan policymakers. XAI memastikan bahwa sistem AI tidak hanya akurat tetapi juga dapat dipercaya dan dapat ditindaklanjuti.

bayangkan bahwa anda seorang educator yang mencoba mengidentifikasi mengapa beberapa student unggul sementara yang lain kesulitan. anda ingin membandingkan siswa tertentu dengan siswa lain, mengungkapkan faktor-faktor keberhasilan utama dan menentukan pola mana yang mempengaruhi kinerja akademis. model AI traditional memberikan prediksi, tetapi jarang memberikan penjelasan. disinilah Explainable AI berperan.

disini anda akan menggunakan IBM AI Explainability 360 (AIX360), perangkat lengkap yang menawarkan teknik-teknik penjelasan canggih untuk menginterpretasikan model machine learning. dengan menerapkan XAI pada datasets kinerja student, saya bertujuan untuk mengidentifikasi profil student representatif **(prototypes)** yang paling meringkas dataset dan memahami karakteristik yang menentukan siswa yang berhasil atau gagal.

In [2]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://www.mdpi.com/electronics/electronics-12-01020/article_deploy/html/images/electronics-12-01020-g002.png"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 1100px; height: auto;"></div>')

---
## 1.1 Why is Relevant Now?
---

XAI akan menciptakan lingkungan transparan tempat users dapat memahami dan mempercayai keputusan yang dibuat AI.

Bisnis menggunakan perangkat AI untuk meningkatkan kinerja dan membuat keputusan yang lebih baik. namun, selain mendapatkan manfaat dari hasil, memahami cara kerjanya juga penting. kurangnya penjelasan mencegah perusahaan membuat skenario what-if yang relevan dan menciptakan masalah kepercayaan kerena mereka tidak memahami bagaimana AI mencapai hasil tertentu. perangkat AI yang lebih kompleks ini bekerja dalam black-box, yang membuat sulit untuk menafsirkan alasan dibalik keputusan.

Sementara manusia dapat menjelaskan AI model yang lebih sederhana seperti decision tree atau logistic regression, model yang lebih akurat seperti neural networks atau random forest adalah black-box models. XAI penting karena menjelaskan AI model black-box, membantu manusia memahami cara kerjanya. ini dapat memperjelas penalaran keputusan tertentu, berbagai hasil, dan kekuatan/kelemahan model. seiring bisnis lebih memahami AI model dan solusinya, XAI membangun kepercayaan antara perusahaan dan AI. teknologi ini memungkinkan bisnis untuk menggunakan AI secara maksimal. [Source1](https://research.aimultiple.com/xai/)

In [3]:
from IPython.display import Image, HTML, display, display_html
source_tautan = "https://ars.els-cdn.com/content/image/1-s2.0-S1566253521001093-gr7.jpg"
HTML(f'<div style="text-align: center;"><img src="{source_tautan}" style="max-width: 1300px; height: auto;"></div>')

---
# 2. Objectives
---

pada proyek ini akan menerapkan metode pengolahan data dan Explainable AI:
  * set up the environment: menginstall perangkat untuk Explainable AI (XAI) pada dataset kinerja siswa.
  * preprocess the dataset: menjelajahi dan menyiapkan data menggunakan teknik enconding dan scaling.
  * build and evaluate a model: melatih model machine learning untuk memprediksi keberhasilan atau kegagalan student.
  * enhance interpretability with Protodash: mengidentifikasi profil student yang representatif dan membandingkan student secara individu dengan prototipe.
  * visualize insights: menggunakan PCA untuk menampilkan hubungan antara student dan prototipe, dan menganalisis pentingnya prototipe.

membangun proyek ini mampu memberikan pengalaman yang luas untuk menerapkan keahlian di XAI dalam memberikan interpretabilitas dalam analisis data pendidikan.

---
# 3. Installing Required Framework:
---

In [4]:
# !pip install aix360
# !pip install openpyxl
# !pip install cvxpy
# !pip instal ecos
# !pip install --no-deps xport

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

from aix360.algorithms.protodash import ProtodashExplainer

---
# 4. Import data and Preprocessing
---

In [6]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/Qu8EfwuQ8OeYxaEuR8ZC9w/student-por.csv'
df = pd.read_csv(url, sep=';')
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  paid        649 non-null    object
 18  activities  649 non-null    object
 19  nursery     649 non-null    object
 20  higher    

The dataset contains data related to Portuguese language performance among secondary school students in Portugal. It includes various demographic, social, and academic attributes, providing a rich source of information for analyzing factors influencing student performance.

The `info()` function provides a concise summary of the dataset, including the number of non-null entries, data types, and memory usage. A breakdown of the dataset follows:

| Column     | Type        | Description                                                                                  | Category Types/Range                                                                                 |
|------------|-------------|----------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------|
| school     | Categorical | The student's school.                                                                        | 'GP' - Gabriel Pereira, 'MS' - Mousinho da Silveira                                           |
| sex        | Categorical | The student's gender.                                                                        | 'F' - Female, 'M' - Male                                                                      |
| age        | Numerical   | The student's age.                                                                           | From 15 to 22                                                                                 |
| address    | Categorical | The student's home address type.                                                             | 'U' - Urban, 'R' - Rural                                                                      |
| famsize    | Categorical | Family size.                                                                                 | 'LE3' - Less or equal to 3, 'GT3' - Greater than 3                                            |
| Pstatus    | Categorical | Parent's cohabitation status.                                                                | 'T' - Living together, 'A' - Apart                                                            |
| Medu       | Numerical   | Mother's education.                                                                          | 0 - None, 1 - Primary, 2 - 5th to 9th grade, 3 - Secondary, 4 - Higher                        |
| Fedu       | Numerical   | Father's education.                                                                          | 0 - None, 1 - Primary, 2 - 5th to 9th grade, 3 - Secondary, 4 - Higher                        |
| Mjob       | Categorical | Mother's job.                                                                                | 'teacher', 'health', 'services', 'at_home', 'other'                                           |
| Fjob       | Categorical | Father's job.                                                                                | 'teacher', 'health', 'services', 'at_home', 'other'                                           |
| reason     | Categorical | Reason for choosing this school.                                                             | 'home', 'reputation', 'course', 'other'                                                      |
| guardian   | Categorical | Student's guardian.                                                                          | 'mother', 'father', 'other'                                                                   |
| traveltime | Numerical   | Home to school travel time.                                                                  | 1 - <15 min, 2 - 15–30 min, 3 - 30–60 min, 4 - >60 min                                       |
| studytime  | Numerical   | Weekly study time.                                                                           | 1 - <2 hours, 2 - 2–5 hours, 3 - 5–10 hours, 4 - >10 hours                                   |
| failures   | Numerical   | Number of past class failures.                                                               | 0 - No failures, 1 to 3 - Actual failures, 4 - More than 3                                    |
| schoolsup  | Categorical | Extra educational support.                                                                   | 'yes', 'no'                                                                                   |
| famsup     | Categorical | Family educational support.                                                                  | 'yes', 'no'                                                                                   |
| paid       | Categorical | Extra paid classes within the course subject.                                                | 'yes', 'no'                                                                                   |
| activities | Categorical | Extra-curricular activities.                                                                 | 'yes', 'no'                                                                                   |
| nursery    | Categorical | Attended nursery school.                                                                     | 'yes', 'no'                                                                                   |
| higher     | Categorical | Wants to take higher education.                                                              | 'yes', 'no'                                                                                   |
| internet   | Categorical | Internet access at home.                                                                     | 'yes', 'no'                                                                                   |
| romantic   | Categorical | In a romantic relationship.                                                                  | 'yes', 'no'                                                                                   |
| famrel     | Numerical   | Quality of family relationships.                                                             | 1 - Very bad to 5 - Excellent                                                                 |
| freetime   | Numerical   | Free time after school.                                                                      | 1 - Very low to 5 - Very high                                                                 |
| goout      | Numerical   | Going out with friends.                                                                      | 1 - Very low to 5 - Very high                                                                 |
| Dalc       | Numerical   | Workday alcohol consumption.                                                                 | 1 - Very low to 5 - Very high                                                                 |
| Walc       | Numerical   | Weekend alcohol consumption.                                                                 | 1 - Very low to 5 - Very high                                                                 |
| health     | Numerical   | Current health status.                                                                       | 1 - Very bad to 5 - Very good                                                                 |
| absences   | Numerical   | Number of school absences.                                                                   | From 0 to 93                                                                                 |
| G1         | Numerical   | First-period grade.                                                                          | From 0 to 20                                                                                 |
| G2         | Numerical   | Second-period grade.                                                                         | From 0 to 20                                                                                 |
| G3         | Numerical   | Final grade.                                                                                 | From 0 to 20                                                                                 |


dataset ini memberikan gambara menyeluruh tentang aspek sosio-demografis dan akademis yang mempengaruhi kinerja bahasa portugis, ideal untuk studi pendidikan dan psikologi.

variabel target akan menganalisis **G3 (Nilai Akhir)** karena mewakili kinerja akademis siswa secara keseluruhan dan berfungsi sebagai hasil yang ingin di analisis dan prediksi.

---
# 5. Checking For Missing Values And Duplicate Value
---

In [8]:
max(df.isnull().max())

False

In [9]:
max(df.duplicated())

False

---
# 6. Model Training and Evaluation
---

untuk memulai training model, pertama-tama dipisahkan variabel target G3 dari dataset. fitur-fitur tersebut kemudian digunakan untuk memprediksi variabel target. selanjutnya, membagi dataset menjadi data training dengan data testing untuk mengevaluasi kinerja model.

variabel target G3 dibinerisasi (misalnya, lulus/gagal berdasarkan nilai threshold) untuk menyederhanakan masalah menjadi tugas klasifikasi. peendekatan ini memudahkan untuk menginterpretasikan hasil dan memahami prediksi model sambil berfokus pada apakah seseorang siswa memenuhi kriteria kinerja tertentu.

penting untuk dicatat bahwa nilai threshold untuk menentukan lulus atau gagal dapat bervariasi berdasarkan kasus penggunaan tertentu atau standar regional. nilai threshold saat ini merupakan asumsi yang dibuat untuk kesederhanaan. kamu dianjurkan untuk bereksperimen dengan nilai threshold yang berbeda untuk mengamati bagaimana kinerja model dan output Protodash Explainer berubaj sesuai dengan perubahan tersebut.

In [10]:
X = df.drop(columns=['G3'])   # drop kolom target yaitu G3 dari set fitur
y = (df['G3'] > 8).astype(int)    # binerisasi G3: 1 untuk lulus (G3 > 0), 0 untuk gagal.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
X_train

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
116,GP,M,15,U,GT3,T,4,4,other,teacher,...,no,4,4,3,1,1,2,4,16,15
146,GP,M,16,U,LE3,T,1,1,services,other,...,yes,4,4,4,1,3,5,0,10,10
358,GP,F,18,U,LE3,T,4,3,health,services,...,no,3,2,4,1,4,1,8,12,12
134,GP,F,17,U,LE3,T,2,2,other,other,...,yes,3,4,4,1,3,5,2,13,12
584,MS,F,17,R,GT3,T,0,0,at_home,other,...,no,4,4,3,1,1,5,0,10,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,MS,F,17,R,GT3,T,2,1,at_home,other,...,yes,5,5,3,1,1,3,2,9,10
102,GP,M,15,U,GT3,T,4,4,services,other,...,no,5,3,3,1,1,5,2,12,13
156,GP,F,15,U,GT3,T,1,1,other,services,...,no,4,4,2,1,2,5,0,12,12
354,GP,F,18,U,GT3,T,2,1,services,other,...,no,5,3,3,1,2,1,2,12,12


setelah **train_test_split** mengatur ulang indeks untuk menghindari masalah penyelarasan. artinya, setelah pemisahan, subset mungkin membawa indeks baris asli dari DataFrame induk. misalnya X_train mungkin memiliki baris berlabel [10,15, 21, ...] dan bukan [0,1,2,...] dengan mengatur ulang indeks, kita memastikan bahwa setiap subset memiliki indeks bilangan bulat yang bersih dan berurutan mulai dari 0. kemudian baris 0 dari X_train berkorespondensi persis dengan baris 0 dari y_train.

In [13]:
# atur ulang indeks untuk menghindari masalah penyelarasan
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

---
# 7. Handling Categorical Features
---

dari dataset, kamu dapat mengamati bahwa dataset tersebut berisi campuran fitur kategoris dan numerik. diantara fitur kategoris:

  * berapa bersifat biner (misalnya, schoolsup - yes/no)
  * beberapa sifat ordinal (misalnya, studytime - 1: < 2 jam, 4: > 10 jam)
  * beberapa bersifat multikelas (misalnya, Mjob - guru, health, services, etc)
  * yang lain bersifat nominal (misalnya, alasan - rumah, reputasi, kursus)

penanganan fitur-fitur ini dengan tepat sangat penting untuk memastikan model bekerja secara optimal. untuk mencapainya, akan mengkodekan dan menskalakan fitur-fitur ini menggunakan teknik yang tepat:
  * mengkodekan fitur biner menggunakan OneHotEncoder
    * mengkonversikan variabel biner menjadi nilai numerik (0 dan 1).
  * One-hot Encoder fitur multikelas dengan one-hot:
    * mencegah model mangasumsikan hubungan ordinal dalam fitur nominal.
    * mengembangkan fitur kategoris menjadi kolm biner terpisah untuk setiap kategori.

menskalakan fitur ordinal atau nominal karena Random Forest tidak memerlukan input yang diskalakan. RandoForest terdiri dari decision tree, dan membagi data berdasarkan urutan nilai, bukan besarnya secara tepat. dengan kata lain, jika anda menggunakan transformasi monotonik yang ketat seperti mengalikan semua nilai dengan 2, atau menerapkan fungsi logaritma ke angka positif, urutan relatif titik data tetap sama, sehingga titik pembagi tree tetap sama secara efektif.

misalnya, perhatikan fitur ordinal studytime dengan nilai dari 1 sampai 4. jika anda menerapkan penskalaa non-linear, nilai-nilai tersebut mungkin berubah, tetapi 1 masih kurang dari 2, 2 masih kurang dari 3, dan seterusnya. karena diagram hanya peduli tetntang siapa yang lebih besar atau lebih kecil, pembaginya tidak berubah secara mendasar.

untuk fitur nominal (seperti alasan: rumah, reputasi, kursus), tidak ada urutan alami sama sekali. menetapkan skala numerik tidak membantu. sebagai gantinya, menggunakan OneHotEncoder sehingga setiap kategori dapat ditangani dengan benar oleh model, tanpa berpura-pura ada skala numerik yang berarti.

In [14]:
# menentukan fitur karegoris
binary_features = ['school', 'sex', 'address',
                   'famsize', 'Pstatus', 'schoolsup',
                   'famsup', 'paid', 'activities',
                   'nursery', 'higher', 'internet', 'romantic']
multi_class_features = ['Mjob', 'Fjob', 'reason', 'guardian']

disioni untuk mengkonversi fitur kategoris multikelas kedalam format numerik yang dapat dipahami model. pertama, melatih encoder pada data training, sehingga ia mempelajari kategori mana yang ada. kemudian, menggunakan encoder yang sama untuk mengubah data testing, memastikan bahwa set testing dikodekan hanya berdasarkan kategori yang terlihat selama training.

In [15]:
# menerapkan pengkodean One-hot Encoder secara terpisah ke set training dan testing
# handle_unknown='ignore' digunakan untuk mencegah masalah kategori yang tidak terlihat
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

# sesuaikan dengan data training dan ubah set training dan testing
X_train_encoded = ohe.fit_transform(X_train[multi_class_features])
X_test_encoded = ohe.transform(X_test[multi_class_features])

# mengkonversikan ke dataframe dengan nama kolom yang sesuai
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=ohe.get_feature_names_out(multi_class_features), index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=ohe.get_feature_names_out(multi_class_features), index=X_test.index)


In [16]:
# drop fitur multikelas asli dan ganti dengan fitur yang dikodekan
X_train = X_train.drop(columns=multi_class_features).reset_index(drop=True)
X_test = X_test.drop(columns=multi_class_features).reset_index(drop=True)

X_train = pd.concat([X_train.reset_index(drop=True), X_train_encoded_df.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_encoded_df.reset_index(drop=True)], axis=1)

In [17]:
# menerapkan One-hot-encoder secara terpisah untuk fitur biner
binary_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
binary_encoder

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [18]:
X_train_binary = binary_encoder.fit_transform(X_train[binary_features])
X_train_binary

array([[0., 1., 1., ..., 1., 0., 0.],
       [0., 1., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 1., 1., 0.],
       ...,
       [0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 1., ..., 1., 1., 0.]])

In [19]:
X_test_binary = binary_encoder.transform(X_test[binary_features])
X_test_binary

array([[0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 0.]])

In [20]:
# mengubah fitur biner menjadi DataFrame dengan nama kolom
X_train_binary_df = pd.DataFrame(X_train_binary, columns=binary_encoder.get_feature_names_out(binary_features), index=X_train.index)
X_train_binary_df

,school_MS,sex_M,address_U,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
515,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
516,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
517,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0


In [21]:
X_test_binary_df = pd.DataFrame(X_test_binary, columns=binary_encoder.get_feature_names_out(binary_features), index=X_test.index)
X_test_binary_df

,school_MS,sex_M,address_U,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
126,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
127,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
128,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [22]:
# drop kolom kategori biner asli dan mengganti dengan yang dikodekan
X_train = X_train.drop(columns=binary_features).reset_index(drop=True)
X_test = X_test.drop(columns=binary_features).reset_index(drop=True)

In [23]:
X_train = pd.concat([X_train.reset_index(drop=True), X_train_binary_df.reset_index(drop=True)], axis=1)
X_train

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,15,4,4,2,2,0,4,4,3,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,16,1,1,1,2,2,4,4,4,1,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,18,4,3,2,1,0,3,2,4,1,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,17,2,2,1,1,0,3,4,4,1,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
4,17,0,0,2,1,0,4,4,3,1,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,17,2,1,3,1,0,5,5,3,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
515,15,4,4,1,1,0,5,3,3,1,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
516,15,1,1,1,2,0,4,4,2,1,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
517,18,2,1,2,2,0,5,3,3,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0


In [24]:
X_test = pd.concat([X_test.reset_index(drop=True), X_test_binary_df.reset_index(drop=True)], axis=1)
X_test

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,16,4,3,1,1,0,5,4,5,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,16,3,4,1,2,0,5,3,3,1,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,19,1,1,1,2,2,4,4,3,3,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
3,17,0,1,2,1,0,2,4,4,3,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,17,2,2,2,2,0,5,1,3,1,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,15,1,3,3,1,0,4,2,4,3,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
126,17,4,4,1,3,0,5,4,4,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
127,17,3,4,1,3,0,4,3,4,2,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
128,18,4,3,1,3,0,5,3,2,1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [25]:
print(X_train.columns)

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3',
       'Pstatus_T', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')


---
# 8. Encoded Data:
---

setelah menerapkan teknik pengkodean, fitur kategoris dalam dataset telah diubah menjadi representasi numerik yang sesuai untuk model machine learning. berikut adalah tabel yang menggambarkan bagaimana beberapa fitur kategoris telah dipetakan dari kategori aslinya ke nilai yang dikodekan.

| Feature      | Original Categories                         | Encoded Values             |
|--------------|--------------------------------------------|----------------------------|
| school       | 'GP', 'MS'                                 | `school_MS` → 1 for 'MS', 0 for 'GP' |
| sex          | 'F', 'M'                                   | `sex_M` → 1 for 'M', 0 for 'F' |
| address      | 'U' (Urban), 'R' (Rural)                  | `address_U` → 1 for 'U', 0 for 'R' |
| famsize      | 'LE3' (≤3 family members), 'GT3' (>3)     | `famsize_LE3` → 1 for 'LE3', 0 for 'GT3' |
| Pstatus      | 'T' (Together), 'A' (Apart)               | `Pstatus_T` → 1 for 'T', 0 for 'A' |
| schoolsup    | 'yes', 'no'                                | `schoolsup_yes` → 1 for 'yes', 0 for 'no' |
| internet     | 'yes', 'no'                                | `internet_yes` → 1 for 'yes', 0 for 'no' |
| higher       | 'yes', 'no'                                | `higher_yes` → 1 for 'yes', 0 for 'no' |
| Mjob         | 'teacher', 'health', 'services', 'other', 'at_home' | `Mjob_teacher`, `Mjob_health`, etc. (One-hot encoded) |
| Fjob         | 'teacher', 'health', 'services', 'other', 'at_home' | `Fjob_teacher`, `Fjob_health`, etc. (One-hot encoded) |
| reason       | 'home', 'reputation', 'course', 'other'   | `reason_home`, `reason_reputation`, etc. (One-hot encoded) |
| guardian     | 'mother', 'father', 'other'               | `guardian_mother`, `guardian_other` (One-hot encoded) |

fitur numerik (**age**, **Medu**, **Fedu**, **traveltime**, **studytime**, dll) tetap tidak berubah karena RandomForest tidak memerlukan penskalaan fitur. pengodean memastikan bahwa variabel kategoris berkontribusi dengan tepat pada model training tanpa memperkenalkan hubungan ordinal buatan.

---
# 9. Training Model: Random Forest Classifier
---

In [26]:
rf= RandomForestClassifier(max_depth=6, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=6, random_state=42)

In [27]:
rf_y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, rf_y_pred)
print(f'Model Accuracy Score: {accuracy:.3f}')
print('\n')

print('Classification Report:\n', classification_report(y_test, rf_y_pred))

Model Accuracy Score: 0.938


Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.85      0.73        13
           1       0.98      0.95      0.97       117

    accuracy                           0.94       130
   macro avg       0.81      0.90      0.85       130
weighted avg       0.95      0.94      0.94       130



model terebut mencapai akurasi sebesar 94% yang mencerminkan kinerja yang kuat dalam memprediksi hasil student. model tersebut menunjukkan presisi tinggi 98% dan recall 95% untuk kategori pass category, sementara kategori Fail memiliki recall yang cukup baik 85% tetapi presisi sedang 65%. hal ini menunjukkan bahwa model tersebut efektif dalam mengidentifikasi sebagian besar student yang gagal, meskipun beberapa hasil false positive tetap ada.

---
# 10. ProtoDash Explainer:
---

Protodash Explainer adalah algoritma canggih dari pustaka AIX360 yang dirancang untuk meningkatkan interpretabilitas dengan mengidentifikasi contoh prototipe atau profil penggunaan representatif dari dataset. pendekatan ini memungkinkan pengguna untuk lebih memahami titik data tertentu atau sekelompok titik data dengan membandingkannya dengan contoh serupa dalam data training. pendekatan ini sangat berguna dalam aplikasi seperti education, healtcare, atau loan approvals, dimana pemahaman alasan dibalik keputusan sangat penting.

---
## 10.1. What is ProtodashExplainer?
---
protodashexplainer adalah algoritma yang memilih sekumpulan prototipe representatif dari dataset untuk menjelaskan titik data atau subkumpulan data tertentu. dengan membandingkan contoh target dengan prototipe ini, pengguna dapat memperoleh wawasan tentang pola dan karakteristik dalam dataset yang mempengaruhi keputusan model.

---
## 10.2. How Does ProtoDashExplainer Work?
---

* prototype selection:
  * algoritma memilih sejumlah prototipe tertentu dengan meminimalkan perbedaan rata-rata (MMD) anatara instans target (atau subset) dan prototipe. ini memastikan prototipe mewakili distribusi data keseluruhan atau subset tertentu yang dianalisis.
  * Maximum Mean Discrepancy (MMD) adalah cara untuk mengukur seberapa berbeda dua dataset dengan melihat perbedaan rata-rata antara fitur-fiturnya.

* Optimasi Greedy dengan bobot:
  * algoritma optimasi greedy digunakan untuk menemukan prototipe terbaik secara berulang. setiap prototipe yang dipilih diberi bobot, yang menunjukkan kemiripannya dengan instans target atau subset. bobot yang lebih tinggi menandakan kemiripan yang lebih dekat, sementara bobot yang lebih rendah menyoroti prototipe yang kurang relevan.

* explaining individual and group profiles:
  * titik data individu: dengan menemukan prototipe yang sangat cocok dengan instans target.
  * kelompok atau subset: dengan meringkas karakteristik seluruh kelompok, seperti siswa yang gagal atau pelamar berisiko tinggi.
  * pengguna dapat menyesuaikan jumlah prototipe (m) berdasarkan tingkat ketelitian penjelasan yang diinginkan.

proyek ini akan menerapkan protodash explainer untuk mengidentifikasi prototipe yang mewakili student yang gagal dan membandingkannya dengan student pilihan. pendekatan ini akan menyoroti persamaan dan perbedaan utama, membantu kita untuk memahami profil student dalam kaitannya dengan group.

---
## 10.3. Understanding Failing Students Through Prototypes
--

di bagian ini, akan menampilkan penggunaan ProtoDashExplainer untuk menemukan contoh student representatif (prototipe) yang gagal dan membandingkannya dengan student tertentu dari rangkaian tes. ini membantu dalam memahami apa yang membuat student serupa dengan student lainnya yang gagal dan memberikan wawasan tentang profil student tersebut.

  * Identify Failing Students
    * disini akan dilakukan filter data training untuk membuat subset student yang gagal **(failing_student)**, dimana label target **(y_train)** adalah **0 (yang menunjukkan kegagalan)**.
  * Choose a Test student:
    * memilih satu student dari data testing (chosen_student) untuk membandingkan dengan student yang gagal.
  * Run ProtoDashExplainer:
    * penjelasan memilih 5 prototipe (m=5) dari student yang gagal. prototipe ini adalah yang paling mirip dengan student yang dipilih.
    * setiap prototipe diberi bobot untuk menunjukkan seberapa dekat kecocokannya dengan siswa yang dipilih.

In [28]:
# menentukan siswa yang gagal berdasarkan prediksi model atau label (misalnya, siswa dengan label '0')
failing_students = X_train[y_train.reset_index(drop=True) == 0]      # memastikan penyelarasan dengan mengatur ulang indeks y_train

# pastikan hanya student yang gagal yang disertakan
print("Number Of Failing Students In X_train:", failing_students.shape[0])

# menginformasikan bahwa semua syudent di failing_students memang diberi label "Gagal" (0)
print('\nUnique Labels In y_train For failing_students: ', y_train.loc[failing_students.index].unique())

# pilih student dari dataset testing untuk dibandingkan dengan student yang gagal
chosen_student = X_test.iloc[0].values.reshape(1,-1)   # ambil siswa pertama dari set testing

Number Of Failing Students In X_train: 52

Unique Labels In y_train For failing_students:  [0]


In [29]:
# inisialisasi ProtoDashExplainer
explainer = ProtodashExplainer()
explainer

In [30]:
# menemukan sampel representatif (prototipe) dari student yang gagal, dengan m=5 untuk jumlah prototipe
weights, indices, set_values = explainer.explain(chosen_student, failing_students.values, m=6)

Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point


In [31]:
# cetak indeks student gagal yang representatif dan bobotnya
print('Representatif Samples (Prototypes) For Failing Students: ', indices)
print('Weights Of Prototypes: ', weights)

Representatif Samples (Prototypes) For Failing Students:  [40 39 44 42 23 18]
Weights Of Prototypes:  [-7.37337438e-22  4.54169238e-22  1.07059071e-01  3.96113894e-01
  3.96772349e-01  3.25638577e-01]


Output dari ProtoDashExplainer menyediakan tiga komponen kunci yang membantu dalam memahami kesamaan antara applicant target dan prototipe terpilih:
  * weights (W): bobot yang lebih tinggi menunjukkan kesamaan yang lebih  besar antara target dan prototipe terpilih.
  * Indeks(S): setiap indeks memetakan ke profil data training, yang memungkinkan untuk memeriksa nilai fitur yang mendorong kesamaan.
  * Nilai MMD(setValues): skor ini mengukur perbedaan keseluruhan antara target dan prototipe, nilai yang lebih rendah menunjukkan kecocokan yang lebih dekat dengan target.

In [32]:
# memilih student yang dipilih
sample = pd.DataFrame.from_records(chosen_student, columns=X_train.columns)
sample.transpose()

,0
age,16.0
Medu,4.0
Fedu,3.0
traveltime,1.0
studytime,1.0
failures,0.0
famrel,5.0
freetime,4.0
goout,5.0
Dalc,1.0


disini, ada hasil analisis prototipe dengan mengekstrak nilai fitur dari dataset training. setiap prototipe kemudian diberikan label lulus atau gagal berdasarkan nilai targetnya di y_train. untuk memahami kepentingan relatifnya, mengoptimalkan bobot yang ditetapkan untuk setiap prototipe, sehingga jelas mana yang paling mirip dengan siswa yang dipilih. akhirnya, informasi ini dikompilasi menjadi DataFrame, yang menampilkan nilai fitur prototipe, label yang diprediksi dan bobot yang dinormalisasi untuk memudahkan interpretasi.

In [33]:
# mengambil nilai fitur dari prototipe
# memetakan indeks dari failing_students kembali ke X_train untuk memastikan aligment.
prototype_indices_in_X_train = failing_students.iloc[indices].index
prototype_indices_in_X_train

Index([415, 409, 456, 425, 229, 192], dtype='int64')

In [34]:
# menggunakan indeks yang dipetakan untuk mengambil baris yang benar dari X_train
# memastikan bahwa prototipe selaras dengan dataset asli
prototypes_df = pd.DataFrame.from_records(X_train.loc[prototype_indices_in_X_train].values, columns=X_train.columns)
prototypes_df

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,16.0,4.0,4.0,1.0,2.0,0.0,4.0,3.0,4.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
1,18.0,2.0,2.0,1.0,1.0,1.0,4.0,4.0,3.0,2.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2,18.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,5.0,2.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,16.0,4.0,4.0,1.0,1.0,0.0,3.0,3.0,2.0,2.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
4,15.0,2.0,3.0,1.0,2.0,0.0,4.0,4.0,4.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
5,16.0,2.0,1.0,1.0,1.0,0.0,5.0,4.0,3.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0


In [35]:
# dapatkan label untuk setiap prototipe (1=lulus, 0=gagal)
# inisialisasi daftar kosong untuk menyimpan label yang diprediksi
predicted_labels = []

# lakukan looping melalui proses indeks prototipe yang dipetakan dan tentukan labelnya
# gunakan y_train untuk mengambil label yang benar untuk prototipr
for idx in prototype_indices_in_X_train:
  if y_train.loc[idx] == 0:     # check apakah labelnya kosong
    predicted_labels.append('Fail')
  else:      #  jika tidak, labelnya adalah 1 (lulus)
    predicted_labels.append('Pass')

# menambahkan nama kelas yang diprediksi ke DataFrame prototipe
prototypes_df['Prediction'] = predicted_labels
prototypes_df

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,Prediction
0,16.0,4.0,4.0,1.0,2.0,0.0,4.0,3.0,4.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Fail
1,18.0,2.0,2.0,1.0,1.0,1.0,4.0,4.0,3.0,2.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,Fail
2,18.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,5.0,2.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,Fail
3,16.0,4.0,4.0,1.0,1.0,0.0,3.0,3.0,2.0,2.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,Fail
4,15.0,2.0,3.0,1.0,2.0,0.0,4.0,4.0,4.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,Fail
5,16.0,2.0,1.0,1.0,1.0,0.0,5.0,4.0,3.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Fail


In [36]:
# menormalisasikan bobot untuk menunjukkan kesamaan relatif setiap prototipe dengan student yang dipilih
# bobot dinormalisasi agar berjumlah 1 untuk memudahkan interpretasi
prototypes_df['Weight'] = np.around(weights, 6)/np.sum(np.around(weights, 6))
prototypes_df

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,Prediction,Weight
0,16.0,4.0,4.0,1.0,2.0,0.0,4.0,3.0,4.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Fail,-0.000000
1,18.0,2.0,2.0,1.0,1.0,1.0,4.0,4.0,3.0,2.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,Fail,0.000000
2,18.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,5.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,Fail,0.087353
3,16.0,4.0,4.0,1.0,1.0,0.0,3.0,3.0,2.0,2.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,Fail,0.323204
4,15.0,2.0,3.0,1.0,2.0,0.0,4.0,4.0,4.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,Fail,0.323741
5,16.0,2.0,1.0,1.0,1.0,0.0,5.0,4.0,3.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Fail,0.265701


In [37]:
# mengatur ulang indeks DataFrame untuk output yang lebih bersih
prototypes_df = prototypes_df.reset_index(drop=True)
prototypes_df

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,Prediction,Weight
0,16.0,4.0,4.0,1.0,2.0,0.0,4.0,3.0,4.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Fail,-0.000000
1,18.0,2.0,2.0,1.0,1.0,1.0,4.0,4.0,3.0,2.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,Fail,0.000000
2,18.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,5.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,Fail,0.087353
3,16.0,4.0,4.0,1.0,1.0,0.0,3.0,3.0,2.0,2.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,Fail,0.323204
4,15.0,2.0,3.0,1.0,2.0,0.0,4.0,4.0,4.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,Fail,0.323741
5,16.0,2.0,1.0,1.0,1.0,0.0,5.0,4.0,3.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Fail,0.265701


In [38]:
# menunjukkan prototipe DataFrame dengan kolom prediksi dan bobot
# mengubah urutan DataFrame agar lebih mudah dibaca
print(prototypes_df.transpose())

                      0     1         2         3         4         5
age                16.0  18.0      18.0      16.0      15.0      16.0
Medu                4.0   2.0       1.0       4.0       2.0       2.0
Fedu                4.0   2.0       1.0       4.0       3.0       1.0
traveltime          1.0   1.0       1.0       1.0       1.0       1.0
studytime           2.0   1.0       1.0       1.0       2.0       1.0
failures            0.0   1.0       2.0       0.0       0.0       0.0
famrel              4.0   4.0       2.0       3.0       4.0       5.0
freetime            3.0   4.0       3.0       3.0       4.0       4.0
goout               4.0   3.0       5.0       2.0       4.0       3.0
Dalc                1.0   2.0       2.0       2.0       1.0       1.0
Walc                2.0   2.0       5.0       1.0       1.0       1.0
health              3.0   1.0       4.0       5.0       1.0       5.0
absences            4.0  26.0       0.0      16.0       0.0       2.0
G1                  

tabel diatas menunjukkan prototipe yang dipilih oleh ProtoDashExplainer, yang membantu menjelaskan bagaimana student yang dipilih dibandingkan dengan student lain dalam dataset. setiap prototipe mewakili student sungguhan, yang dijelaskan oleh nilai fitur mereka (misalnya, usia, waktu belajar, jumlah anggota keluarga), label prediksi (lulus atau gagal), dan bobot. bobot menunjukkan seberapa mirip detiap prototipe dengan student yang dipilih, dengan bobot yang lebih tinggi berarti kemiripan yang lebih kuat.

misalnya, prototipe 4 memiliki bobot tertinggi (0.323741), yang berarti paling mewakili siswa yang dipilih, sebaliknya, prototipe 0 memiliki bobot yang jauh lebih rendah (-0.0) yang menunjukkan tidak ada kesamaan.

hasil ini membantu dalam memahami karakteristik student yang dipilih dalam kaitannya dengan student lain:
  * feature similarities: kita dapat mengamati bahwa student yang dipilih memiliki ciri-ciri tertentu misalnya, waktu belajar atau akses internet dengan prototipe berbobot tinggi).
  * prediction comparrison: kolom prediksi menunjukkan bahwa semua prototipe dalam contoh ini diberi label gagal, yang memberikan konteks tentang bagaimana profil student yang dipilih selaras dengan student yang gagal.
  * weight interpretation: prototipe berbobot lebih tinggi memberikan perbandingan dan wawasan yang lebih bermakna tentang alasan student yang dipilih menerima prediksi mereka.

---
# 11. PCA Visualization of Prototypes and Chosen Student:
---

grafik dibawah ini menggambarkan visualisasi PCA (Principal Component Analysis) dari data training, prototipe, dan student yang dipilih. PCA mengurangi dimensionalitas dataset menjadi dua komponen, sehingga memudahkan analisis visual hubungan antara titik data. dengan memetakan data student berdimensi tinggi ke dalam ruang berdimensi renda, PCA membantu menyoroti persamaan dan perbedaan diantara student, sehingga kita dapat melihat bagaimana student yang dipilih selaras dengan prototipe representatif.

What is Happening in the Graph?

  * Training data (Blue Points):
    * mewakili semua student dalam dataset pelatihan baik yang lulus maupun tidak lulus.
    * penyebaran mereka dalam ruang 2D ditentukan oleh fitur yang ditransformasikan PCA.
  * Protoypes (Red Points):
    * dipilih oleh ProtoDashExplainer untuk mewakili profil student utama.
    * diposisikan untuk meringkas karakteristik student yang paling berpengaruh dalam dataset.
  * Chosen Student (Green "X"):
    * mewakili student yang sedang dianalisis.
    * dibandingkan dengan prototipe untuk memahami bagaimana prototipe tersebut berhubungan dengan student yang gagal dalam set pelatihan.

In [39]:
# mengkonversikan chosen_student menjadi DataFrame dengan nama kolom yang cocok.
chosen_student_df = pd.DataFrame(chosen_student, columns=X_train.columns)
chosen_student_df

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,16.0,4.0,3.0,1.0,1.0,0.0,5.0,4.0,5.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [40]:
# menyesuaikan PCA pada data training.
pca = PCA(n_components=2)
pca

PCA(n_components=2)

In [41]:
x_pca = pca.fit_transform(X_train)     # sesuaikan PCA pada data training dan ubahlah
x_pca

array([[-2.07546482,  5.65080626],
       [-2.34507163, -3.66743197],
       [ 3.92985717,  2.61226234],
       ...,
       [-3.70574375, -0.90320229],
       [-1.73565475, -0.00885445],
       [ 7.39855468,  3.94504742]])

In [42]:
# petakan indeks dari failing_student kembali ke X_train
prototype_indices_in_X_train = failing_students.iloc[indices].index
prototype_indices_in_X_train

Index([415, 409, 456, 425, 229, 192], dtype='int64')

In [43]:
# mentransformasikan prototipe dan student yang dipilih menggunakan PCA
prototype_pca = pca.transform(X_train.loc[prototype_indices_in_X_train])   # transformasi prototypes
prototype_pca

array([[ 2.20180239, -3.9470669 ],
       [22.81829828,  3.73747305],
       [-1.93094092, -3.86810305],
       [12.59317999,  2.04503308],
       [-1.23955764, -6.32806937],
       [-0.18381634, -4.02634383]])

In [44]:
chosen_pca = pca.transform(chosen_student_df)   # transform the chosen student
chosen_pca

array([[1.60975323, 4.8226488 ]])

In [45]:
y_train = np.random.choice([0,1], size=len(X_train))  # ingin membuat label target (y_train) secara acak berupa 0 atau 1, dengan jumlah sama seperti data training X_train
# membuat daftar ID siswa dalam format "Student 0", "Student 1", ..., sebanyak jumlah data X_train
student_ids = [F'STUDENT {i}' for i in range(len(X_train))]
# mengambil proporsi varian yang dijelaskan oleh komponen utama pertama dan kedua dari hasil PCA, lalu dikonversi ke persen
pc1_var, pc2_var = pca.explained_variance_ratio_[:2] * 100

# membuat dataframe baru (df_pca) yang berisi informasi dua komponen utama dari hasil PCA, label acak, dan ID siswa
df_pca = pd.DataFrame({'PC1': x_pca[:,0],
                       'PC2': x_pca[:,1],
                       'Label': y_train,
                       'ID': student_ids})
df_pca

,PC1,PC2,Label,ID
0,-2.075465,5.650806,0,STUDENT 0
1,-2.345072,-3.667432,1,STUDENT 1
2,3.929857,2.612262,1,STUDENT 2
3,-1.872341,0.514012,1,STUDENT 3
4,-2.759131,-3.126129,0,STUDENT 4
...,...,...,...,...
514,-0.409782,-3.429273,0,STUDENT 514
515,-2.252449,1.035767,0,STUDENT 515
516,-3.705744,-0.903202,1,STUDENT 516
517,-1.735655,-0.008854,0,STUDENT 517


In [46]:
# Mapping label numerik ke nama kelas dan warna untuk visualisasi
label_map = {0: 'Fail', 1:'Pass'}   # mapping label ke nama kelas
color_map = {0: 'tomato', 1:'dodgerblue'}   # mapping label ke warna
training_traces = []   # list untuk menyimpan trace data training berdasarkan kelas

# trace: training data by class
for label in sorted(df_pca['Label'].unique()):
  df_class = df_pca[df_pca['Label'] == label]    # filter berdasarkan label kelas (0 atau 1)
  trace = go.Scattergl(x=df_class['PC1'],
                       y=df_class['PC2'],
                       mode='markers',
                       marker=dict(size=7,
                                   color=color_map[label],   # warna berdasarkan kelas
                                   opacity=0.6),
                       name=f'Class {label_map[label]}',   # nama di legenda
                       text=df_class['ID'],                       # ID Student hover
                       hovertemplate=("<b>%{text}</b><br>"
                                      "Class: " + label_map[label]+ "<br>"
                                      "PC1: %{x:.2f}<br>"
                                      "PC2: %{y:.2f}<extra></extra>"))
  training_traces.append(trace)    # simpan trace ke list

# traces: Prototypes
trace_prototypes = go.Scattergl(x=prototype_pca[:,0],
                                y=prototype_pca[:,1],
                                mode='markers+text',
                                marker=dict(size=16,
                                            color='crimson',
                                            line=dict(width=1.5,
                                                      color='black')),
                                text=[f'P{i}' for i in range(len(prototype_pca))],
                                textposition='top right',
                                name='Prototypes',
                                hovertemplate="<b>Prototype %{text}</b><br>PC1: %{x:.2f}<br>PC2: %{y:.2f}<extra></extra>")


# traces: Chosen Students
trace_chosen = go.Scattergl(x=chosen_pca[:,0],
                            y=chosen_pca[:,1],
                            mode='markers+text',
                            marker=dict(size=20, color='mediumseagreen', symbol='x', line=dict(width=2, color='black')),
                            text=[f'C{i}' for i in range(len(chosen_pca))],
                            textposition='bottom left',
                            name='Chosen Student',
                            hovertemplate="<b>Chosen Student %{text}</b><br>PC1: %{x:.2f}<br>PC2: %{y:.2f}<extra></extra>")


# layout Stylish
layout = go.Layout(title=dict(text='PCA Projection With Class Coloring, Prototypes, and Chosen Student',
                              x=0.5,
                              font=dict(size=22)),
                   xaxis=dict(title=f'Principal Component 1 ({pc1_var:.1f}%)',
                              showgrid=True,
                              gridcolor='lightgray',
                              zeroline=False,
                              ticks='outside'),
                   yaxis=dict(title=f'Principal Component 2 ({pc2_var:.1f}%)',
                              showgrid=True,
                              gridcolor='lightgray',
                              zeroline=False,
                              ticks='outside'),
                   legend=dict(bgcolor='rgba(255,255,255,0.95)',
                               bordercolor='lightgray',
                               borderwidth=1,
                               font=dict(size=12)),
                   plot_bgcolor='white',
                   margin=dict(l=60, r=60, t=80, b=60),
                   width=1100,
                   height=650)

In [47]:
fig = go.Figure(data=training_traces + [trace_prototypes, trace_chosen], layout=layout)
fig.update_layout(hovermode='closest', dragmode='pan')
fig.show()

---
## 11.1 Observation From The PCA Visualization:
---

* **student yang dipilih tanda X** tampak sedikit terpisah dari banyak titik training, yang menunjukkan karakteristik yang berbeda atau profil yang kurang umum.
* **prototipe (lingkaran merah)** tersebar diantara data training, yang menunjukkan bahwa profil student yang gagal bervariasi dan tidak terbatas pada satu kluster.
* beberapa prototipe terletak lebih dekat dengan student terpilih, yang menyiaratkan kesamaan yang lebih kuat dalam nilai fitur dan berpotensi lebih berpengaruh pada penjelasan siswa yang dipilih.
* distribusi titik class lulus dan class gagal menunjukkan bentuk umum populasi student, dengan wilayah yang lebih padat tempat banyak student memiliki karakteristik yang tumpang tindih.
* prototipe yang terletak lebih dekat di ruang PCA umumnya menawarkan lebih banyak wawasan tentang mengapa student yang dipilih diberi label serupa, sedangkan yang lebih jauh memiliki kesamaan yang lebih rendah.

dengan menganalisis visualisasi ini, memperoleh wawasan tentang kluster student yang serupa, ciri-ciri yang mengelompokkan mereka dan bagaimana student yang dipilih dibandingkan dengan dataset yang lebih luas.

In [48]:
w , idx, _ = explainer.explain(X_train.values, X_train.values, m=11)
print("Representatif Samples (Prototypes):", idx)
print("Weights of Prototypes:", w)

Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Polishing not needed - no active set detected at optimal point
Representatif Samples (Prototypes): [332 391 487 181 506 238 302 275 100 137 131]
Weights of Prototypes: [ 1.99384820e-01  7.48161574e-02  8.65793387e-02  7.40145774e-02
 -2.11536456e-21  6.40595786e-02  1.43991089e-01  1.07122087e-01
  1.31921453e-01  1.37719633e-02  7.58985208e-02]


In [49]:
prototype_profiles = X_train.iloc[idx]
print("\nPrototype Profiles (Top 3 Representative Students):")
print(prototype_profiles)


Prototype Profiles (Top 3 Representative Students):
     age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  \
332   16     2     3           1          2         0       4         4   
391   17     4     2           2          4         0       4         2   
487   19     1     1           2          1         1       5         5   
181   17     3     3           1          2         0       5         3   
506   18     4     4           2          2         0       4         3   
238   18     4     4           1          1         0       1         4   
302   19     2     1           2          3         1       4         3   
275   15     2     1           4          1         0       4         5   
100   17     4     4           1          2         0       5         2   
137   18     1     2           2          1         0       4         1   
131   18     3     2           1          2         0       3         1   

     goout  Dalc  ...  famsize_LE3  Pstatus_T 